<a href="https://colab.research.google.com/github/galenzo17/AI-personal-test/blob/main/CustomerService.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install websockets openai asyncio nest_asyncio --quiet

import asyncio
import websockets
import json
import nest_asyncio

# Esto permite que asyncio corra en Colab
nest_asyncio.apply()

# Define la URL de conexión de la API Realtime
REALTIME_API_URL = "wss://api.openai.com/v1/realtime"

# Coloca aquí tu clave de la API de OpenAI
API_KEY = "tu-api-key"

# Función para enviar una solicitud de chat
async def connect_realtime():
    # Crea el encabezado con la autenticación
    headers = {
        "Authorization": f"Bearer {API_KEY}"
    }

    try:
        # Abre una conexión WebSocket
        async with websockets.connect(REALTIME_API_URL, extra_headers=headers) as websocket:
            # Primer mensaje de bienvenida
            welcome_message = {
                "model": "gpt-4o-realtime-preview",
                "messages": [
                    {"role": "system", "content": "You are a customer support agent for a pharmaceutical company. Respond to questions about medications, dosages, side effects, and availability in a professional manner."},
                    {"role": "user", "content": "Hello, I need help with a medication I purchased."}
                ]
            }

            # Envía el mensaje de bienvenida
            await websocket.send(json.dumps(welcome_message))

            # Espera la respuesta del modelo
            while True:
                try:
                    response = await websocket.recv()
                    response_data = json.loads(response)

                    # Verifica si la respuesta tiene la estructura esperada
                    if 'choices' in response_data and response_data['choices']:
                        print(f"Response: {response_data['choices'][0]['message']['content']}")
                    else:
                        print("Unexpected response format:", response_data)

                    # Toma nueva entrada del usuario
                    user_input = input("You: ")
                    if user_input.lower() in ["exit", "quit"]:
                        break

                    # Enviar nuevo mensaje basado en el input del usuario
                    new_message = {
                        "model": "gpt-4o-realtime-preview",
                        "messages": [
                            {"role": "user", "content": user_input}
                        ]
                    }
                    await websocket.send(json.dumps(new_message))

                except json.JSONDecodeError:
                    print("Error decoding the response from the server. Retrying...")
                except KeyError:
                    print("Key not found in response. Check the response structure.")

    except websockets.exceptions.ConnectionClosedError as e:
        print(f"Connection was closed unexpectedly: {e}")
    except Exception as e:
        print(f"An error occurred: {e}")

# Ejecuta la conexión
await connect_realtime()

Unexpected response format: {'type': 'error', 'event_id': 'event_AKaEGgPfXhnRuocxA9sph', 'error': {'type': 'invalid_request_error', 'code': 'invalid_api_key', 'message': 'Incorrect API key provided: tu-api-key. You can find your API key at https://platform.openai.com/account/api-keys.', 'param': None, 'event_id': None}}
You: Hola
Connection was closed unexpectedly: received 3000 (registered) invalid_request_error.invalid_api_key; then sent 3000 (registered) invalid_request_error.invalid_api_key
